In [ ]:
# !pip install --user --upgrade pip

# !pip install pandas
!pip install spectrum
!pip install pyhrv
# !pip install cvxopt

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)

cvx_eda_path = os.path.abspath(os.path.join('..', '..', 'scvxOPT'))
sys.path.append(cvx_eda_path)

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import tools.data_reader_ascertain as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift

# import warnings
# warnings.filterwarnings(
#     "ignore", 
#     category=RuntimeWarning
# )

In [23]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

from biosppy.signals import ecg
import heartpy as hp

convert_sr = False
is_clean_ecg = True
metrics = "bpm"
fs = 256.0

for clip in dr.CLIPS:
    data = []
    for s in dr.SUBJECTS:
        ecg_df = dr.read_ecg(s, clip, "left")
        ecg_signal = ecg_df["ecg"]
        # ecg_signal = preprocessing.clean_ecg(ecg_signal, fs).to_numpy().flatten()
        out = ecg.ecg(signal=ecg_signal, sampling_rate=fs, show=False)
        ecg_signal = out["filtered"]
        bpm = out["heart_rate"].tolist()

        # print(np.mean(bpm))

        # working_data, measures = hp.process_segmentwise(ecg_signal, fs, segment_width=55, segment_overlap=5/55)
        # print(measures)
        # print(np.mean(measures["bpm"]))

        bpm.insert(0, int(s))
        data.append(bpm)
        
    file_name = os.path.join(dr.Paths.METRICS, f"bpm_Clip{clip}.csv")
    df = pd.DataFrame(data=data)
    df.to_csv(file_name)
        



In [26]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import biosppy
from biosppy.signals import ecg

import pyhrv
import pyhrv.time_domain as td

convert_sr = False
is_clean_ecg = True
fs = 256.0

for clip in dr.CLIPS:
# for clip in dr.CLIPS[0:2]:
    metrics_dict = {
        "rmssd": [],
        # "ibi": [],
        "sdnn": []
    }
    for s in dr.SUBJECTS:
    # for s in dr.SUBJECTS[0:5]:
        ecg_df = dr.read_ecg(s, clip, "left")
        ecg_signal = ecg_df["ecg"]
        # ecg_signal = preprocessing.clean_ecg(ecg_signal, fs).to_numpy().flatten()

        ecg_features = preprocessing.get_ecg_metrics_pyhrv(ecg_signal, fs)

        for metric in list(metrics_dict.keys()):
            feature = ecg_features[metric]
            feature.insert(0, int(s))
            metrics_dict[metric].append(feature)

    # for metric in list(metrics_dict.keys()):
    #     df = pd.DataFrame(metrics_dict[metric])
    #     metrics_dict[metric] = df
        
    # for metric in metrics_dict.keys():
    #     file_name = os.path.join(dr.Paths.METRICS, f"{metric}_Clip{clip}.csv")
    #     metrics_dict[metric].to_csv(file_name)
        



Not enough beats to compute heart rate.


In [ ]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import biosppy
from biosppy.signals import ecg

import heartpy as hp

convert_sr = False
is_clean_ecg = True
fs = 256.0

for clip in dr.CLIPS:
    metrics_dict = {
        "lf_rr": [],
        "hf_rr": [],
    }
    for s in dr.SUBJECTS:
        ecg_df = dr.read_ecg(s, clip, "left")
        ecg_signal = ecg_df["ecg"]
        # ecg_signal = preprocessing.clean_ecg(ecg_signal, fs).to_numpy().flatten()
        out = ecg.ecg(signal=ecg_signal, sampling_rate=fs, show=False)
        ecg_signal = out["filtered"]
        
        lf_rr = preprocessing.get_lf_rr(ecg_signal, fs)
        lf_rr = np.insert(lf_rr, 0, s)
        lf_rr = pd.DataFrame(lf_rr).dropna(axis=1)

        hf_rr = preprocessing.get_hf_rr(ecg_signal, fs)
        hf_rr = np.insert(hf_rr, 0, s)
        hf_rr = pd.DataFrame(hf_rr).dropna(axis=1)

        metrics_dict["lf_rr"].append(lf_rr)
        metrics_dict["hf_rr"].append(hf_rr)
        
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df

    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_Clip{clip}.csv")
        metrics_dict[metric].to_csv(file_name)

In [32]:
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import biosppy
from biosppy.signals import eda

convert_sr = False
is_clean_ecg = True
fs = 128.0

for clip in dr.CLIPS:
    metrics_dict = {
        "mean_SCL": [],
        "SCR_rate": [],
    }
    for s in dr.SUBJECTS:
        eda_df = dr.read_gsr(s, clip, "x")
        eda_signal = eda_df["eda"]
        # ecg_signal = preprocessing.clean_ecg(ecg_signal, fs).to_numpy().flatten()
        out = eda.eda(signal=eda_signal, sampling_rate=fs, show=False)
        eda_signal = out["filtered"]
        
        mean_scl = preprocessing.get_mean_SCL(eda_signal, fs)
        mean_scl = np.insert(mean_scl, 0, s)
        mean_scl = pd.DataFrame(mean_scl).dropna(axis=1)

        scr_rate = preprocessing.get_SCR_rate(eda_signal, fs)
        scr_rate = np.insert(scr_rate, 0, s)
        scr_rate = pd.DataFrame(scr_rate).dropna(axis=1)

        metrics_dict["mean_SCL"].append(mean_scl)
        metrics_dict["SCR_rate"].append(scr_rate)
        
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df

    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_Clip{clip}.csv")
        metrics_dict[metric].to_csv(file_name)